In [556]:
import pandas as pd
import numpy as np
import tensorflow as tf
from os import listdir
from sklearn.model_selection import train_test_split
print ('The version of TensorFlow is {}'.format(tf.__version__))

The version of TensorFlow is 1.4.0


In [557]:
root_path = './data'

appended_data = []
for file_name in listdir(root_path):
    file_path = root_path + '/' + file_name.encode().decode('utf-8')
    data_from_one_csv = pd.read_csv(file_path, skiprows=1)
    appended_data.append(data_from_one_csv)
    
data = pd.concat(appended_data, axis=0)
data = data.drop(columns = ['ACN', 'Date', 'Local Time Of Day', 'Ceiling', 'Callback', 'Callback.1', 'Unnamed: 96'])
data = data.rename(index=str, columns={"Flight Phase": "Flight Phase1"})

## drop the rows with empty synopsis description
data = data[pd.notnull(data['Synopsis'])]

X = data.drop(columns = 'Result')
Y_raw = pd.DataFrame(data['Result'])

processed_Y = []
for index, row in Y_raw.iterrows():
    #print (index, row['Result'])
    outcome = row['Result']
    if type(outcome) == np.float:
        res = 'unknown'
        processed_Y.append([res])
    elif ';' in outcome:
        res = str(outcome).split(';')
        processed_Y.append(res)
    else:
        res = outcome
        processed_Y.append([res])

In [558]:
## compress the number of labels to be predicted --> map result to risk level
rate_nine = ['General Declared Emergency', 'General Physical Injury / Incapacitation', 'Flight Crew Inflight Shutdown', 
             'Air Traffic Control Separated Traffic', 'Aircraft Aircraft Damaged']

rate_seven = ['General Evacuated', 'Flight Crew Landed as Precaution', 'Flight Crew Regained Aircraft Control', 
              'Air Traffic Control Issued Advisory / Alert', 'Flight Crew Landed in Emergency Condition',
              'Flight Crew Landed In Emergency Condition']

rate_five = ['General Work Refused', 'Flight Crew Became Reoriented', 'Flight Crew Diverted', 
             'Flight Crew Executed Go Around / Missed Approach', 
             'Flight Crew Overcame Equipment Problem', 'Flight Crew Rejected Takeoff', 'Flight Crew Took Evasive Action', 
             'Air Traffic Control Issued New Clearance']

rate_three = ['General Maintenance Action', 'General Flight Cancelled / Delayed', 
              'General Release Refused / Aircraft Not Accepted', 
              'Flight Crew Overrode Automation', 'Flight Crew FLC Overrode Automation',
              'Flight Crew Exited Penetrated Airspace', 
              'Flight Crew Requested ATC Assistance / Clarification', 'Flight Crew Landed As Precaution',
              'Flight Crew Returned To Clearance', 'Flight Crew Returned To Departure Airport',
              'Aircraft Automation Overrode Flight Crew']

rate_one = ['General Police / Security Involved', 'Flight Crew Returned To Gate', 'Aircraft Equipment Problem Dissipated', 
            'unknown', 'Air Traffic Control Provided Assistance',
            'General None Reported / Taken', 'Flight Crew FLC complied w / Automation / Advisory']

def risk_quantification(val):
    min_risk = []
    for i in range(len(val)):
        item = val[i].lstrip() ## remove the space at the start of each item
        if item in rate_nine:
            min_risk.append(5)
        elif item in rate_seven:
            min_risk.append(4)
        elif item in rate_five:
            min_risk.append(3)
        elif item in rate_three:
            min_risk.append(2)
        elif item in rate_one:
            min_risk.append(1)
    return max(min_risk)


Y_ = []
for i in range(len(processed_Y)):
    if len(processed_Y[i]) > 1:
        val = risk_quantification(processed_Y[i])
        Y_.append(val)
    else:
        item_val = " ".join(processed_Y[i]) ## convert a list to a string
        if item_val in rate_nine:
            Y_.append(5)
        elif item_val in rate_seven:
            Y_.append(4)
        elif item_val in rate_five:
            Y_.append(3)
        elif item_val in rate_three:
            Y_.append(2)
        elif item_val in rate_one:
            Y_.append(1)
        else:
            print (Y['Result'][i])

outcomes = np.asarray(Y_)
Y_pred = pd.DataFrame(Y_, index = X.index, columns = ['Result'])
unique, counts = np.unique(outcomes, return_counts=True)
print (unique, counts)

[1 2 3 4 5] [16508  8636 18841  8261 12327]


In [559]:
data_rev = X.copy(deep=True)
data_rev['Result'] = Y_pred

## Upsampling the minority categories

In [560]:
from sklearn.utils import resample

df_majority_1 = data_rev[data_rev['Result']==1]
df_majority_3 = data_rev[data_rev['Result']==3]
df_minority_2 = data_rev[data_rev['Result']==2]
df_minority_4 = data_rev[data_rev['Result']==4]
df_minority_5 = data_rev[data_rev['Result']==5]

# Upsample minority class
df_minority_2_upsampled = resample(df_minority_2, 
                                 replace=True,     # sample with replacement
                                 n_samples=18000,    # to match majority class
                                 random_state=123) # reproducible results
df_minority_4_upsampled = resample(df_minority_4, 
                                 replace=True,     # sample with replacement
                                 n_samples=18000,    # to match majority class
                                 random_state=123) # reproducible results
df_minority_5_upsampled = resample(df_minority_5, 
                                 replace=True,     # sample with replacement
                                 n_samples=18000,    # to match majority class
                                 random_state=123) # reproducible results

df_upsampled = pd.concat([df_majority_1, df_majority_3, df_minority_2_upsampled, df_minority_4_upsampled, 
                          df_minority_5_upsampled])

df_upsampled['Result'].value_counts()

X = df_upsampled.drop(columns = 'Result')
Y_pred = df_upsampled['Result']

unique, counts = np.unique(Y_pred, return_counts=True)
print ('After the upsampling, the number of each item is: \n')
print (unique)
print (counts)

After the upsampling, the number of each item is: 

[1 2 3 4 5]
[16508 18000 18841 18000 18000]


## Split the data

In [561]:
from sklearn.model_selection import train_test_split

test_size_ratio = 0.2
random_split_seed = 100
X_train, X_test, Y_train, Y_test = train_test_split(X['Synopsis'], Y_pred, test_size = test_size_ratio, 
                                                    random_state = random_split_seed)

## Pipeline: Naive Bayes

In [562]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB(alpha = 1, fit_prior=True)),
                    ])

text_clf.fit(X_train, Y_train)
pred_label = text_clf.predict(X_test)

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 5+1)]
print(classification_report(Y_test, pred_label, target_names=target_names))

             precision    recall  f1-score   support

          1       0.60      0.48      0.54      3304
          2       0.67      0.62      0.64      3623
          3       0.49      0.42      0.45      3831
          4       0.53      0.58      0.55      3533
          5       0.58      0.77      0.66      3579

avg / total       0.57      0.57      0.57     17870



## Model 1: Support Vector Machine with Linear Kernel

In [563]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='epsilon_insensitive', penalty='l2',
                                            alpha=1e-5, random_state=40,
                                            max_iter=10, tol=None)),
                    ])


parameters = {'clf__loss': ['epsilon_insensitive', 'hinge', 'log', 'huber', 'modified_huber', 'perceptron', 
                            'squared_loss', 'squared_epsilon_insensitive', 'squared_hinge'],
              'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3, 1e-4, 1e-5),
              'clf__penalty': ['l1', 'l2', 'elasticnet'],
              'clf__max_iter': (10, 20, 30, 40, 50, 60, 70, 80, 90, 100)
 }

optimal_parameters = {'clf__loss': ['modified_huber'],
              'vect__ngram_range':  [(1, 2)],
              'tfidf__use_idf': [True],
              'clf__alpha': [1e-5],
              'clf__penalty': ['elasticnet'],
              'clf__max_iter': [80],
 }

gs_clf = GridSearchCV(text_clf, optimal_parameters, n_jobs=-1)

gs_clf.fit(X_train, Y_train)
pred_label_SVM = gs_clf.predict(X_test)

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 6)]
print(classification_report(Y_test, pred_label_SVM, target_names=target_names))

             precision    recall  f1-score   support

          1       0.70      0.57      0.63      3304
          2       0.84      0.92      0.88      3623
          3       0.66      0.58      0.62      3831
          4       0.78      0.90      0.84      3533
          5       0.86      0.91      0.88      3579

avg / total       0.77      0.78      0.77     17870



In [564]:
print ('Accuracy: ', np.sum(np.equal(Y_test, pred_label).astype(int))/20367)
print ('The best set of parameters is \n', gs_clf.best_params_)

Accuracy:  0.5035105808415574
The best set of parameters is 
 {'clf__alpha': 1e-05, 'clf__loss': 'modified_huber', 'clf__max_iter': 80, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


## Model 2: Processing categorical data

In [565]:
## change column names
new_col_name = []
for col in X.columns:
    #print(type(col))
    new_col_name.append(col.replace('/ ', '').replace(' ', '_'))
    
X.columns = new_col_name

## output the headers from the csv file
X.keys()

data_type = []
for item_name in X.keys():
    data_type.append(type(X[item_name][0]))

print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('The unique data types across all the items are:', set(data_type))
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

for item_name in X.keys():
    ## find the number of NaN in this item
    no = np.sum(X[item_name].isna().astype(int))
    #print ('The number of {} with value equal to NaN is {}'.format(item_name, no))
    
    ## Replace the missing value with corresponding values
    if no > 0:
        if type(X[item_name][0]) == np.float64:
            X[item_name].fillna(-1, inplace = True)
        else:
            X[item_name].fillna('unknown', inplace = True)
X['Crew_Size'].head()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The unique data types across all the items are: {<class 'numpy.float64'>, <class 'float'>, <class 'str'>}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


8     2.0
13    2.0
14    2.0
17    2.0
19    2.0
Name: Crew_Size, dtype: float64

In [566]:
## Location
Locale_Reference = tf.feature_column.categorical_column_with_hash_bucket('Locale_Reference', 
                                                                         hash_bucket_size = len(set(X['Locale_Reference'])))
State_Reference = tf.feature_column.categorical_column_with_hash_bucket('State_Reference', 
                                                                        hash_bucket_size = len(set(X['State_Reference'])))


## Environment
Flight_Conditions = tf.feature_column.categorical_column_with_hash_bucket('Flight_Conditions', 
                                                                hash_bucket_size = len(set(X['State_Reference'])))
Weather_Elements_Visibility = tf.feature_column.categorical_column_with_hash_bucket('Weather_Elements_Visibility', 
                                                            hash_bucket_size = len(set(X['Weather_Elements_Visibility'])))
Work_Environment_Factor = tf.feature_column.categorical_column_with_hash_bucket('Work_Environment_Factor', 
                                                            hash_bucket_size = len(set(X['Work_Environment_Factor'])))
Light = tf.feature_column.categorical_column_with_hash_bucket('Light', hash_bucket_size = 
                                                              len(set(X['Work_Environment_Factor'])))


## Aircraft
ATC_Advisory = tf.feature_column.categorical_column_with_hash_bucket('ATC_Advisory', 
                                                            hash_bucket_size = len(set(X['ATC_Advisory'])))
Aircraft_Operator = tf.feature_column.categorical_column_with_hash_bucket('Aircraft_Operator', 
                                                                hash_bucket_size = len(set(X['Aircraft_Operator'])))
Make_Model_Name = tf.feature_column.categorical_column_with_hash_bucket('Make_Model_Name', 
                                                            hash_bucket_size = len(set(X['Make_Model_Name'])))
Crew_Size = tf.feature_column.numeric_column('Crew_Size', [1])
Flight_Plan = tf.feature_column.categorical_column_with_hash_bucket('Flight_Plan', 
                                                            hash_bucket_size = len(set(X['Flight_Plan'])))
Mission = tf.feature_column.categorical_column_with_hash_bucket('Mission', 
                                                                hash_bucket_size = len(set(X['Mission'])))
Flight_Phase1 = tf.feature_column.categorical_column_with_hash_bucket('Flight_Phase1', 
                                                                      hash_bucket_size = len(set(X['Flight_Phase1'])))
Route_In_Use = tf.feature_column.categorical_column_with_hash_bucket('Route_In_Use', 
                                                                     hash_bucket_size = len(set(X['Route_In_Use'])))
Airspace = tf.feature_column.categorical_column_with_hash_bucket('Airspace', 
                                                                 hash_bucket_size = len(set(X['Airspace'])))

## Component
Aircraft_Component = tf.feature_column.categorical_column_with_hash_bucket('Aircraft_Component', 
                                                             hash_bucket_size = len(set(X['Aircraft_Component'])))
Manufacturer = tf.feature_column.categorical_column_with_hash_bucket('Manufacturer', 
                                                        hash_bucket_size = len(set(X['Manufacturer'])))

## Person
Location_Of_Person = tf.feature_column.categorical_column_with_hash_bucket('Location_Of_Person', 
                                                                hash_bucket_size = len(set(X['Location_Of_Person'])))
Location_In_Aircraft = tf.feature_column.categorical_column_with_hash_bucket('Location_In_Aircraft',
                                                            hash_bucket_size = len(set(X['Location_In_Aircraft'])))
Reporter_Organization = tf.feature_column.categorical_column_with_hash_bucket('Reporter_Organization',
                                                            hash_bucket_size = len(set(X['Reporter_Organization'])))
Function = tf.feature_column.categorical_column_with_hash_bucket('Function', hash_bucket_size = len(set(X['Function'])))
Qualification = tf.feature_column.categorical_column_with_hash_bucket('Qualification', 
                                                                      hash_bucket_size = len(set(X['Qualification'])))
Human_Factors = tf.feature_column.categorical_column_with_hash_bucket('Human_Factors', 
                                                                      hash_bucket_size = len(set(X['Human_Factors'])))

## Events
Anomaly = tf.feature_column.categorical_column_with_hash_bucket('Anomaly', 
                                                                hash_bucket_size = len(set(X['Anomaly'])))
Detector = tf.feature_column.categorical_column_with_hash_bucket('Detector', 
                                                                 hash_bucket_size = len(set(X['Detector'])))
When_Detected = tf.feature_column.categorical_column_with_hash_bucket('When_Detected', 
                                                                      hash_bucket_size = len(set(X['When_Detected'])))
Were_Passengers_Involved_In_Event = tf.feature_column.categorical_column_with_hash_bucket('Were_Passengers_Involved_In_Event',
                                                    hash_bucket_size = len(set(X['Were_Passengers_Involved_In_Event'])))

## Assessments
Contributing_Factors_Situations = tf.feature_column.categorical_column_with_hash_bucket('Contributing_Factors_Situations', 
                                                   hash_bucket_size = len(set(X['Contributing_Factors_Situations'])))
Primary_Problem = tf.feature_column.categorical_column_with_hash_bucket('Primary_Problem', 
                                                        hash_bucket_size = len(set(X['Primary_Problem'])))

## Place
Locale_Reference = tf.feature_column.embedding_column(Locale_Reference, len(set(X['Locale_Reference'])))
State_Reference = tf.feature_column.embedding_column(State_Reference, len(set(X['State_Reference'])))


## Environment
Flight_Conditions = tf.feature_column.embedding_column(Flight_Conditions,  len(set(X['Flight_Conditions'])))
Weather_Elements_Visibility = tf.feature_column.embedding_column(Weather_Elements_Visibility,  
                                                                 len(set(X['Weather_Elements_Visibility'])))
Work_Environment_Factor = tf.feature_column.embedding_column(Work_Environment_Factor,  len(set(X['Work_Environment_Factor'])))
Light = tf.feature_column.embedding_column(Light, len(set(X['Light'])))


## Aircraft
ATC_Advisory = tf.feature_column.embedding_column(ATC_Advisory, len(set(X['ATC_Advisory'])))
Aircraft_Operator = tf.feature_column.embedding_column(Aircraft_Operator, len(set(X['Aircraft_Operator'])))
Make_Model_Name = tf.feature_column.embedding_column(Make_Model_Name, len(set(X['Make_Model_Name'])))
Flight_Plan = tf.feature_column.embedding_column(Flight_Plan, len(set(X['Flight_Plan'])))
Mission = tf.feature_column.embedding_column(Mission, len(set(X['Mission'])))
Flight_Phase1 = tf.feature_column.embedding_column(Flight_Phase1, len(set(X['Flight_Phase1'])))
Route_In_Use = tf.feature_column.embedding_column(Route_In_Use, len(set(X['Route_In_Use'])))
Airspace = tf.feature_column.embedding_column(Airspace, len(set(X['Airspace'])))

## Component
Aircraft_Component = tf.feature_column.embedding_column(Aircraft_Component, len(set(X['Aircraft_Component'])))
Manufacturer = tf.feature_column.embedding_column(Manufacturer, len(set(X['Manufacturer'])))

## Person
Location_Of_Person = tf.feature_column.embedding_column(Location_Of_Person, len(set(X['Location_Of_Person'])))
Location_In_Aircraft = tf.feature_column.embedding_column(Location_In_Aircraft, len(set(X['Location_In_Aircraft'])))
Reporter_Organization = tf.feature_column.embedding_column(Reporter_Organization, len(set(X['Reporter_Organization'])))
Function = tf.feature_column.embedding_column(Function, len(set(X['Function'])))
Qualification = tf.feature_column.embedding_column(Qualification, len(set(X['Qualification'])))
Human_Factors = tf.feature_column.embedding_column(Human_Factors, len(set(X['Human_Factors'])))

## Events
Anomaly = tf.feature_column.embedding_column(Anomaly, len(set(X['Anomaly'])))
Detector = tf.feature_column.embedding_column(Detector, len(set(X['Detector'])))
When_Detected = tf.feature_column.embedding_column(When_Detected, len(set(X['When_Detected'])))
Were_Passengers_Involved_In_Event = tf.feature_column.embedding_column(Were_Passengers_Involved_In_Event,
                                                                       len(set(X['Were_Passengers_Involved_In_Event'])))

## Assessments
Contributing_Factors_Situations = tf.feature_column.embedding_column(Contributing_Factors_Situations,
                                                                     len(set(X['Contributing_Factors_Situations'])))
Primary_Problem = tf.feature_column.embedding_column(Primary_Problem, len(set(X['Primary_Problem'])))

In [567]:
from sklearn.model_selection import train_test_split

X_sub = X[['Locale_Reference', 'State_Reference', 'Flight_Conditions', 'Weather_Elements_Visibility', 
            'Work_Environment_Factor', 'Light', 'ATC_Advisory', 'Aircraft_Operator', 'Make_Model_Name', 
            'Crew_Size', 'Flight_Plan', 'Mission', 'Flight_Phase1',
            'Route_In_Use','Airspace', 'Aircraft_Component', 'Manufacturer', 'Location_Of_Person', 'Location_In_Aircraft',
            'Reporter_Organization', 'Function', 'Qualification', 'Human_Factors', 'Anomaly', 'Detector', 'When_Detected',
            'Were_Passengers_Involved_In_Event', 'Contributing_Factors_Situations', 'Primary_Problem']]

X_train, X_test, Y_train, Y_test = train_test_split(X_sub, Y_pred, test_size = test_size_ratio, 
                                                    random_state = random_split_seed)

In [568]:
label = []
number_models = 10
for i in range(number_models):
    print ('Train the {} model, please keep waiting !!!'.format(i+1))
    print ('\n')
    
    X_train_set, X_test_tmp, Y_train_set, Y_test_tmp = train_test_split(X_train, Y_train, test_size = 0.2, random_state = i)

    ## define input function
    input_func = tf.estimator.inputs.pandas_input_fn(x = X_train_set, y = Y_train_set, batch_size = 500, 
                                                        num_epochs = 600, shuffle = True)

    ## define the feature columns
    feat_cols = [Locale_Reference, State_Reference, Flight_Conditions, Weather_Elements_Visibility, Work_Environment_Factor, 
                     Light, ATC_Advisory, Aircraft_Operator, Make_Model_Name, Crew_Size, Flight_Plan, Mission, Flight_Phase1, 
                     Route_In_Use, Airspace, Aircraft_Component, Manufacturer, Location_Of_Person, Location_In_Aircraft, 
                     Reporter_Organization, Function, Qualification, Human_Factors, Anomaly, Detector, When_Detected, 
                     Were_Passengers_Involved_In_Event, Contributing_Factors_Situations, Primary_Problem]

    ## build the model
    model = tf.estimator.DNNClassifier(hidden_units = [40, 40, 40, 40, 40, 40, 40, 40], feature_columns = feat_cols,
                                           n_classes = 6, optimizer = tf.train.AdamOptimizer(learning_rate = 0.001))
        
    ## train the model
    model.train(input_fn = input_func, steps = 4000)
    
    
    ## make predictions
    eval_input = tf.estimator.inputs.pandas_input_fn(x = X_test, shuffle = False)
    prediction = list(model.predict(eval_input))

    pred_label = [int(pred['class_ids']) for pred in prediction]
    
    label.append(pred_label)

Train the 1 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmp_vmnn173', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016B19E96AC8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp_vmnn173\model.ckpt.
INFO:tensorflow:loss = 895.6406, step = 1
INFO:tensorflow:global_step/sec: 0.692538
INFO:tensorflow:loss = 554.6471, step = 101 (144.430 sec)
INFO:tensorflow:global_step/sec: 0.681187

INFO:tensorflow:loss = 163.54889, step = 901 (157.736 sec)
INFO:tensorflow:global_step/sec: 0.695928
INFO:tensorflow:loss = 135.29486, step = 1001 (143.566 sec)
INFO:tensorflow:global_step/sec: 0.69543
INFO:tensorflow:loss = 132.26419, step = 1101 (143.792 sec)
INFO:tensorflow:global_step/sec: 0.698025
INFO:tensorflow:loss = 148.837, step = 1201 (143.261 sec)
INFO:tensorflow:Saving checkpoints for 1223 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpt6dfv0qv\model.ckpt.
INFO:tensorflow:global_step/sec: 0.639292
INFO:tensorflow:loss = 107.77811, step = 1301 (156.429 sec)
INFO:tensorflow:global_step/sec: 0.698688
INFO:tensorflow:loss = 101.585556, step = 1401 (143.122 sec)
INFO:tensorflow:global_step/sec: 0.69878
INFO:tensorflow:loss = 118.066216, step = 1501 (143.102 sec)
INFO:tensorflow:global_step/sec: 0.699192
INFO:tensorflow:loss = 80.32904, step = 1601 (143.024 sec)
INFO:tensorflow:Saving checkpoints for 1634 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpt6dfv0qv\model.ckpt.
INFO:tenso

INFO:tensorflow:global_step/sec: 0.622423
INFO:tensorflow:loss = 78.77127, step = 2501 (160.675 sec)
INFO:tensorflow:global_step/sec: 0.694398
INFO:tensorflow:loss = 90.62584, step = 2601 (144.017 sec)
INFO:tensorflow:global_step/sec: 0.696926
INFO:tensorflow:loss = 77.40028, step = 2701 (143.473 sec)
INFO:tensorflow:global_step/sec: 0.695974
INFO:tensorflow:loss = 51.571808, step = 2801 (143.681 sec)
INFO:tensorflow:Saving checkpoints for 2845 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp8z65bygc\model.ckpt.
INFO:tensorflow:global_step/sec: 0.623347
INFO:tensorflow:loss = 78.78998, step = 2901 (160.427 sec)
INFO:tensorflow:global_step/sec: 0.696616
INFO:tensorflow:loss = 86.59249, step = 3001 (143.553 sec)
INFO:tensorflow:global_step/sec: 0.696734
INFO:tensorflow:loss = 93.85217, step = 3101 (143.524 sec)
INFO:tensorflow:global_step/sec: 0.697015
INFO:tensorflow:loss = 72.27592, step = 3201 (143.479 sec)
INFO:tensorflow:Saving checkpoints for 3251 into C:\Users\ZHANGX~1\AppData\Local\

Train the 5 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmp5w9ifm90', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016B023A8EB8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp5w9ifm90\model.ckpt.
INFO:tensorflow:loss = 895.87756, step = 1
INFO:tensorflow:global_step/sec: 0.688499
INFO:tensorflow:loss = 611.86194, step = 101 (145.275 sec)
INFO:tensorflow:global_step/sec: 0.6992

INFO:tensorflow:loss = 175.49884, step = 901 (158.659 sec)
INFO:tensorflow:global_step/sec: 0.694022
INFO:tensorflow:loss = 178.1132, step = 1001 (144.090 sec)
INFO:tensorflow:global_step/sec: 0.695895
INFO:tensorflow:loss = 169.477, step = 1101 (143.714 sec)
INFO:tensorflow:global_step/sec: 0.695384
INFO:tensorflow:loss = 141.09348, step = 1201 (143.789 sec)
INFO:tensorflow:Saving checkpoints for 1223 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpjzeoqzmv\model.ckpt.
INFO:tensorflow:global_step/sec: 0.631361
INFO:tensorflow:loss = 101.30677, step = 1301 (158.394 sec)
INFO:tensorflow:global_step/sec: 0.696692
INFO:tensorflow:loss = 103.029305, step = 1401 (143.546 sec)
INFO:tensorflow:global_step/sec: 0.697239
INFO:tensorflow:loss = 111.59061, step = 1501 (143.420 sec)
INFO:tensorflow:global_step/sec: 0.694168
INFO:tensorflow:loss = 106.143585, step = 1601 (144.054 sec)
INFO:tensorflow:Saving checkpoints for 1631 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpjzeoqzmv\model.ckpt.
INFO:ten

INFO:tensorflow:loss = 92.91165, step = 2501 (160.228 sec)
INFO:tensorflow:global_step/sec: 0.69312
INFO:tensorflow:loss = 79.780045, step = 2601 (144.277 sec)
INFO:tensorflow:global_step/sec: 0.698647
INFO:tensorflow:loss = 68.02444, step = 2701 (143.122 sec)
INFO:tensorflow:global_step/sec: 0.701471
INFO:tensorflow:loss = 81.15653, step = 2801 (142.564 sec)
INFO:tensorflow:Saving checkpoints for 2868 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpbect_pcs\model.ckpt.
INFO:tensorflow:global_step/sec: 0.63554
INFO:tensorflow:loss = 60.695126, step = 2901 (157.344 sec)
INFO:tensorflow:global_step/sec: 0.700164
INFO:tensorflow:loss = 61.784054, step = 3001 (142.818 sec)
INFO:tensorflow:global_step/sec: 0.696231
INFO:tensorflow:loss = 39.342247, step = 3101 (143.630 sec)
INFO:tensorflow:global_step/sec: 0.701406
INFO:tensorflow:loss = 67.02978, step = 3201 (142.573 sec)
INFO:tensorflow:Saving checkpoints for 3278 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpbect_pcs\model.ckpt.
INFO:tensorf

Train the 9 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmpn2y7xq4y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016B15E29978>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpn2y7xq4y\model.ckpt.
INFO:tensorflow:loss = 895.5149, step = 1
INFO:tensorflow:global_step/sec: 0.693462
INFO:tensorflow:loss = 526.39935, step = 101 (144.244 sec)
INFO:tensorflow:global_step/sec: 0.69816

INFO:tensorflow:loss = 168.90057, step = 901 (157.463 sec)
INFO:tensorflow:global_step/sec: 0.698495
INFO:tensorflow:loss = 140.80461, step = 1001 (143.191 sec)
INFO:tensorflow:global_step/sec: 0.697316
INFO:tensorflow:loss = 125.92228, step = 1101 (143.378 sec)
INFO:tensorflow:global_step/sec: 0.700319
INFO:tensorflow:loss = 151.59064, step = 1201 (142.798 sec)
INFO:tensorflow:Saving checkpoints for 1229 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp_b4r_lsg\model.ckpt.
INFO:tensorflow:global_step/sec: 0.633597
INFO:tensorflow:loss = 134.7743, step = 1301 (157.839 sec)
INFO:tensorflow:global_step/sec: 0.698711
INFO:tensorflow:loss = 107.25649, step = 1401 (143.111 sec)
INFO:tensorflow:global_step/sec: 0.698704
INFO:tensorflow:loss = 111.84253, step = 1501 (143.121 sec)
INFO:tensorflow:global_step/sec: 0.698227
INFO:tensorflow:loss = 101.37254, step = 1601 (143.214 sec)
INFO:tensorflow:Saving checkpoints for 1638 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp_b4r_lsg\model.ckpt.
INFO:ten

In [569]:
ensembel_pred = []
for j in range(len(label[0])):
    x = np.zeros(shape = (len(label), 1)) - 1
    for i in range(len(label)):
        x[i] =  label[i][j]
    (values, counts) = np.unique(x, return_counts=True)
    ind = np.argmax(counts)
    ensembel_pred.append((values[ind]))

ensembel_pred

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 5+1)]
print(classification_report(Y_test, ensembel_pred, target_names=target_names))

             precision    recall  f1-score   support

          1       0.62      0.61      0.62      3304
          2       0.81      0.90      0.85      3623
          3       0.59      0.46      0.52      3831
          4       0.82      0.88      0.85      3533
          5       0.80      0.87      0.83      3579

avg / total       0.73      0.74      0.73     17870



## Hybrid model

In [629]:
dict_count = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
for i in range(len(ensembel_pred)):
    if ensembel_pred[i] == pred_label_SVM[i]:
        dict_count[ensembel_pred[i]] += 1

In [630]:
dict_count

{1: 1810, 2: 3424, 3: 1684, 4: 3196, 5: 3188}

In [677]:
final_pred = []

model_NN = [[0.62, 0.61, 0.62, 3304],
            [0.81, 0.90, 0.85, 3623],
            [0.59, 0.46, 0.52, 3831],
            [0.82, 0.88, 0.85, 3533],
            [0.80, 0.87, 0.83, 3579],
           ]

model_SVM = [[0.70, 0.57, 0.63, 3304],
             [0.84, 0.92, 0.88, 3623],
             [0.66, 0.58, 0.62, 3831],
             [0.78, 0.90, 0.84, 3533],
             [0.86, 0.91, 0.88, 3579],
            ]

total_count = 0
count = 0
for i in range(len(ensembel_pred)):
    if ensembel_pred[i] == pred_label_SVM[i]:
        final_pred.append(pred_label_SVM[i])
    else:
        total_count += 1
        
        prob = np.zeros(shape = 5)
        
        label_ensemble = int(ensembel_pred[i]-1)
        p1 = (model_NN[label_ensemble][3] - dict_count[label_ensemble + 1]) /model_NN[label_ensemble][3] * model_NN[label_ensemble][0]
        prob[label_ensemble] = p1
        
        label_SVM = int(pred_label_SVM[i]-1)
        p2 = (model_SVM[label_SVM][3] - dict_count[label_SVM + 1]) /model_SVM[label_SVM][3] * model_SVM[label_SVM][0]
        prob[label_SVM] = p2
              
        other_labels = list(set(range(1, 6)) - set([label_ensemble + 1, label_SVM + 1]))
        total = 0
        for j in range(len(other_labels)):
            prob[other_labels[j]-1] = (2 - model_NN[other_labels[j]-1][1] - model_SVM[other_labels[j]-1][1])/2
            total += prob[other_labels[j]-1]
        
        for j in range(len(other_labels)):
            prob[other_labels[j]-1] = (1 - p1 - p2) * prob[other_labels[j]-1] /total
        
        if p1 > p2:
            final_pred.append(label_ensemble + 1)
            if label_ensemble + 1 == Y_test[i]:
                count += 1
        else:
            final_pred.append(label_SVM + 1)
            if label_SVM + 1 == Y_test[i]:
                count += 1

p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.046138559205078665
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.046138559205078665
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.3165254237288135

p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.046138559205078665
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.07440135861873762
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.28035108958837773  p2:  0.046138559205078665
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.044490753519183  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.046138

p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.044490753519183  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.078216812906878  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.078216812906878  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.046138559205078665
p1:  0.044490753519183  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.07440135861

p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.078216812906878  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.078216812906878  p2:  0.31652542372881354
p1:  0.078216812906878  p2:  0.09395361832914222
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.

p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.044490753519183  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.078216812906878  p2:  0.316525423728

...

p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.078216812906878  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.078216812906878  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.046138559205078665
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.08739871472478346  p2:  0.07440135861873762
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.04613855920

p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.044490753519183  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.093953618329

p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.09395361832914222
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.08739871472478346  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.369882

p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.046138559205078665
p1:  0.044490753519183  p2:  0.31652542372881354
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.28035108958837773  p2:  0.09395361832914222
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.078216812906878  p2:  0.09395361832914222
p1:  0.078216812906878  p2:  0.31652542372881354
p1:  0.044490753519183  p2:  0.07440135861873762
p1:  0.078216812906878  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544

p1:  0.078216812906878  p2:  0.046138559205078665
p1:  0.33065257113025315  p2:  0.046138559205078665
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.044490753519183  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.31652542372881354
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.08739871472478346  p2:  0.31652542372881354
p1:  0.08739871472478346  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.08739871472478346  p2:  0.046138559205078665
p1:  0.08739871472478346  p2:  0.07440135861873762
p1:  0.28035108958837773  p2:  0.3698825371965544
p1:  0.044490753519183  p2:  0.3698825371965544
p1:  0.33065257113025315  p2:  0.3165254

In [678]:
print(classification_report(Y_test, final_pred, target_names=target_names))

             precision    recall  f1-score   support

          1       0.68      0.58      0.62      3304
          2       0.93      0.88      0.90      3623
          3       0.59      0.72      0.65      3831
          4       0.91      0.86      0.88      3533
          5       0.86      0.88      0.87      3579

avg / total       0.79      0.79      0.79     17870



In [663]:
from sklearn.metrics import confusion_matrix
print ('Confusion matrix: \n', confusion_matrix(Y_test, final_pred))

Confusion matrix: 
 [[2169  145  857   65   68]
 [ 129 3249  168   31   46]
 [ 782  141 2550  213  145]
 [ 105   19  297 3066   46]
 [ 138   63  240   45 3093]]


In [664]:
p1

0.044490753519183

In [665]:
p2

0.3698825371965544

In [666]:
count/total_count

0.5024080560420315

In [578]:
total_count

4568

In [579]:
count

2262